Pobieranie wiadomości z plotek.pl
Zapis do pliku ./data/plotek_{timestamp}.txt

In [1]:
# ! pip install bs4
# ! pip install requests
# ! pip install tqdm
# ! pip install pandas
# ! pip install pyarrow
# ! pip install qgrid
# ! pip install blaze
# ! pip install ipynbname

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm, trange
from datetime import datetime, timezone
from common.webarticle import WebArticle
from common.pl_helper import PolishLocaleHelper
import pandas as pd
import logging
import ipynbname

In [3]:
# settings:
portal_name = 'plotek'
website = 'https://www.plotek.pl/plotek/0,78649.html'
pagination_attributes = '?str='
no_articles_to_scrap = 300
data_dir = '../data'
logging.basicConfig(
    filename=f'../logs/{ipynbname.name()}.ipynb.log',
    encoding='utf-8',
    level=logging.DEBUG
)
ignored_paragraph_starts = ['Więcej o', 'Więcej informacji', 'Więcej ciekawych', 'Więcej ciekawostek', 'Więcej na temat', 'Więcej aktualności',
                            'Więcej artykułów', 'Więcej podobnych', 'Więcej kryminalnych', 'Więcej bieżących', 'Więcej historii', 'Więcej tekstów']

Sprawdźmy plik robots.txt, w czasie pisania tego kodu pozwalał na pobieranie plików z portalu plotek z kategorii news, ograniczenia dotyczyły innych kategorii

In [4]:
resp = requests.get('https://www.plotek.pl/robots.txt')
print(resp.text)

User-agent: *
Disallow: /ot-amp-consent
Disallow: /plotek/0,89428.html?
Disallow: /plotek/0,87207.html?
Disallow: /plotek/0,89890.html?
Disallow: /plotek/0,80228.html?
Disallow: /plotek/0,81806.html?
Disallow: /plotek/0,83251.html?
Disallow: /plotek/0,82863.html?
Disallow: /plotek/0,81708.html?
Disallow: /75224259/

User-agent: Googlebot-News
Disallow: /nowy/
Disallow: /mapa_strony
Disallow: /szukaj/forum
Disallow: /Blox/szukaj
Disallow: /szukaj/poradnik
Disallow: /temat/moto
Disallow: /szukaj/ogrody
Disallow: /temat/gazetapraca
Disallow: /temat/ladnydom
Disallow: /szukaj/plotek
Disallow: /szukaj/polygamia
Disallow: /temat/porady
Disallow: /szukaj/sport
Disallow: /szukaj/gospodarka
Disallow: /temat/wyborcza
Disallow: /szukaj/kuchnia
Disallow: /temat/czterykaty
Disallow: /temat/edukacja
Disallow: /szukaj/edziecko
Disallow: /szukaj/logo
Disallow: /temat/podroze
Disallow: /szukaj/technologie
Disallow: /szukaj/wiadomosci
Disallow: /temat/zdrowie
Disallow: /*/51,
Disallow: /*order=
Disallow

In [5]:
def check_if_ignored_paragraph(text, ignored_paragraphs, operation='startswith'):

    for ignored_paragraph in ignored_paragraphs:
        if operation == 'startswith':
            if text.startswith(ignored_paragraph):
                return True
        else:
            logging.debug(f'check_if_ignored: operation {operation} not implemented')
    return False


In [6]:
def scrap_article_text_plotek(link):
    logging.info('scrap_article_text_plotek:')
    logging.info(f'article link to scrap: {link}')

    page = requests.get(link)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return '', ''

    soup = BeautifulSoup(page.content, 'html.parser')
    paragraphs = soup.find('div', id='gazeta_article_body').find_all('p', class_='art_paragraph')

    text = ''

    for paragraph in paragraphs:
        paragraph_text = paragraph.get_text().strip()
        if check_if_ignored_paragraph(paragraph_text, ignored_paragraph_starts):
            continue
        text += paragraph_text + ' '

    time_string = soup.find('span', class_='article_date').get_text().strip()
    when_published = datetime.strptime(time_string, "%d.%m.%Y %H:%M")

    title_int = soup.find('h1', id='article_title').get_text().strip()
    lead_text_int = soup.find('div', id='gazeta_article_lead').get_text().strip()

    author = soup.find('span', class_='article_author').get_text().strip()
    source = None

    return text, when_published, title_int, lead_text_int, author, source

In [7]:
def scrap_articles_plotek(website):
    logging.info('scrap_articles_plotek:')
    logging.info(f'page with articles to scrap: {website}')

    result = list()
    page = requests.get(website)
    if page.status_code != 200:
        logging.debug(f'received status code: {page.status_code}')
        return result

    soup = BeautifulSoup(page.content, 'html.parser')
    page_content = soup.find('div',class_='index_body').find('div',class_='main_content').find_all('article',class_='article')

    print(f'articles to scrap: {len(page_content)}')

    for article in page_content:

        logging.info(f"article scrapping started: {article}")
        title = article.find('header').get_text().strip()
        lead_text = article.find('p', class_='lead').get_text().strip()
        link = article.find('header').find('a', href=True)['href']
        webart = WebArticle(title, lead_text, link, *scrap_article_text_plotek(link),portal_name)
        logging.info(f'webarticle scrapped: {webart}')
        result.append(webart)

    return result

In [8]:
no_scrapped_articles = 0
page_no=1
scrapped_articles = list()

logging.info(f'no_articles_to_scrap: {no_articles_to_scrap}')

pbar = tqdm(total=no_articles_to_scrap)
while no_scrapped_articles < no_articles_to_scrap:
    logging.info('next page: '+website+pagination_attributes+str(page_no))
    new_articles = scrap_articles_plotek(website+pagination_attributes+str(page_no))
    page_size = len(new_articles)
    logging.info(f'new_articles: {page_size}')
    if no_scrapped_articles + page_size > no_articles_to_scrap:
        page_size = no_articles_to_scrap - no_scrapped_articles
    scrapped_articles.extend(new_articles[:page_size])
    logging.info(f'scrapped new articles: {page_size}')
    no_scrapped_articles += page_size
    pbar.update(page_size)
    logging.info(f'no_scrapped_articles (total): {no_scrapped_articles}')
    page_no += 1
    if len(scrapped_articles) <= 0:
        logging.debug(f'0 articles scrapped!')
        break
pbar.close()

  0%|          | 0/300 [00:00<?, ?it/s]

articles to scrap: 10


  3%|▎         | 10/300 [00:03<01:41,  2.86it/s]

articles to scrap: 10


  7%|▋         | 20/300 [00:07<01:47,  2.61it/s]

...

 93%|█████████▎| 280/300 [01:52<00:07,  2.57it/s]

articles to scrap: 10


 97%|█████████▋| 290/300 [01:56<00:03,  2.61it/s]

articles to scrap: 10


100%|██████████| 300/300 [02:00<00:00,  2.49it/s]


In [9]:
for i in trange(len(scrapped_articles)):
    # print(f'{i}: {scrapped_articles[i].lead_text}')
    print(f'{i}: {len(scrapped_articles[i].text)}')

100%|██████████| 300/300 [00:00<00:00, 30008.61it/s]

0: 801
1: 1390
2: 3858
3: 3353
4: 3099
5: 1359
6: 1099
7: 1246
8: 2565
9: 1370
10: 1688
11: 1648
12: 2898
13: 1496
14: 1496
15: 1075
16: 1919
17: 1230
18: 1651
19: 1139
20: 995
21: 1073
22: 1335
23: 1415
24: 1539
25: 1415
26: 1356
27: 1177
28: 1693
29: 1251
30: 1657
31: 841
32: 1287
33: 945
34: 1507
35: 1143
36: 936
37: 989
38: 1275
39: 1251
40: 1464
41: 930
42: 1411
43: 1786
44: 1485
45: 1354
46: 1054
47: 1017
48: 935
49: 1634
50: 1599
51: 1053
52: 1243
53: 1756
54: 1807
55: 1251
56: 1106
57: 1559
58: 1343
59: 1332
60: 1148
61: 1633
62: 1009
63: 1582
64: 1562
65: 2015
66: 1944
67: 1210
68: 1227
69: 1323
70: 1003
71: 1228
72: 1342
73: 1332
74: 1231
75: 1383
76: 1513
77: 1552
78: 1374
79: 1797
80: 3721
81: 1258
82: 1153
83: 1430
84: 3219
85: 1404
86: 1249
87: 1688
88: 1219
89: 1424
90: 1491
91: 1450
92: 1105
93: 1292
94: 1185
95: 1489
96: 1383
97: 1175
98: 1337
99: 2039
100: 1496
101: 1746
102: 1540
103: 1666
104: 1536
105: 1299
106: 1266
107: 1569
108: 1838
109: 1465
110: 1093
111: 188

In [10]:
plotek_dataframe = pd.DataFrame(article.__dict__ for article in scrapped_articles)
plotek_dataframe

,title,title_int,lead_text,lead_text_int,link,text,when_published,author,source,portal
0,WOŚP. Woźniak-Starak wystawiła na licytację to...,WOŚP. Woźniak-Starak wystawiła na licytację to...,Nadchodzący finał Wielkiej Orkiestry Świąteczn...,Nadchodzący finał Wielkiej Orkiestry Świąteczn...,"https://www.plotek.pl/plotek/7,154063,28020376...",Wielkimi krokami zbliża się 30. Finał Wielkiej...,2022-01-19 22:13:00,Agata Polcyn,None,plotek
1,Gaspard Ulliel zmarł w wyniku tragicznego wypa...,Gaspard Ulliel zmarł w wyniku tragicznego wypa...,"Prokuratura wydała oświadczenie na temat tego,...","Prokuratura wydała oświadczenie na temat tego,...","https://www.plotek.pl/plotek/7,154063,28020408...",19 stycznia świat obiegła informacja o śmierci...,2022-01-19 21:46:00,Joanna Labuda,None,plotek
2,Tilda Swinton jako... Bridget Jones? Nie tylko...,Tilda Swinton jako... Bridget Jones? Nie tylko...,Choć największe kinowe hity zapamiętywane są d...,Choć największe kinowe hity zapamiętywane są d...,"https://www.plotek.pl/plotek/7,154063,28016504...",Kultowe filmy niemal zawsze są połączone są z ...,2022-01-19 21:20:00,Wojciech Zając,None,plotek
3,"Kossakowski zbierał ziemniaki, a Zawadzka prow...","Kossakowski zbierał ziemniaki, a Zawadzka prow...","Osoby, które znamy ze szklanego ekranu, zazwyc...","Osoby, które znamy ze szklanego ekranu, zazwyc...","https://www.plotek.pl/plotek/7,154063,28017851...",72-letnia Vera Wang również zaczęła projektowa...,2022-01-19 20:55:00,Karolina Gawot,None,plotek
4,"Była oczkiem w głowie wuja, odziedziczyła fort...","Była oczkiem w głowie wuja, odziedziczyła fort...","Od najmłodszych lat była obecna w branży mody,...","Od najmłodszych lat była obecna w branży mody,...","https://www.plotek.pl/plotek/7,154063,28017105...",30 czerwca 1986 roku w Mediolanie przyszła na ...,2022-01-19 20:20:00,Monika Tatara,None,plotek
...,...,...,...,...,...,...,...,...,...,...
295,Gala Marki Roku. Blance Lipińskiej bardziej od...,Gala Marki Roku. Blance Lipińskiej bardziej od...,Blanka Lipińska wzięła udział w gali Marki Rok...,Blanka Lipińska wzięła udział w gali Marki Rok...,"https://www.plotek.pl/plotek/7,154063,27917772...",Poza Blanką Lipińską na gali pojawiło się wiel...,2021-12-15 22:40:00,Joanna Labuda,None,plotek
296,Kurdej-Szatan z mężem i dziećmi w bliźniaczych...,Kurdej-Szatan z mężem i dziećmi w bliźniaczych...,15 grudnia gwiazdy pojawiły się na premierze m...,15 grudnia gwiazdy pojawiły się na premierze m...,"https://www.plotek.pl/plotek/7,154063,27917763...",Tłum gwiazd pojawił się 15 grudnia na premierz...,2021-12-15 22:20:00,Anna Goworek,None,plotek
297,Blanka Lipińska wprowadziła się do nowego apar...,Blanka Lipińska wprowadziła się do nowego apar...,Blanka Lipińska wprowadziła się do nowego apar...,Blanka Lipińska wprowadziła się do nowego apar...,"https://www.plotek.pl/plotek/7,78649,27916282,...",Przez ostatnie miesiące Blanka Lipińska remont...,2021-12-15 21:35:00,Joanna Labuda,None,plotek
298,Metamorfoza 77-letniej panny młodej. Powstało ...,Metamorfoza 77-letniej panny młodej. Powstało ...,Odpowiedni sztab specjalistów może czynić cuda...,Odpowiedni sztab specjalistów może czynić cuda...,"https://www.plotek.pl/plotek/7,154063,27915709...",Pod koniec listopada na TikToku pojawiło się n...,2021-12-15 21:15:00,Kamil Zajkowski,None,plotek


In [11]:
timestamp = int((datetime.now(timezone.utc)).timestamp())

In [12]:
plotek_dataframe.to_parquet(f'{data_dir}/{portal_name}_{timestamp}.parquet')
plotek_dataframe.to_json(f'{data_dir}/{portal_name}_{timestamp}.json')

Krótki test - dane pierwszego artykułu

In [13]:
print(f'title:\n  {plotek_dataframe.iloc[0]["title"]}')
print(f'lead_text:\n  {plotek_dataframe.iloc[0]["lead_text"]}')
print(f'lead_text_int:\n  {plotek_dataframe.iloc[0]["lead_text_int"]}')
print(f'link:\n  {plotek_dataframe.iloc[0]["link"]}')
print(f'text:\n  {plotek_dataframe.iloc[0]["text"]}')
print(f'when_published:\n  {plotek_dataframe.iloc[0]["when_published"]}')

title:
  WOŚP. Woźniak-Starak wystawiła na licytację torebki. Wszystkie warte po kilka tysięcy! Jedna to... PODUSZKA
lead_text:
  Nadchodzący finał Wielkiej Orkiestry Świątecznej Pomocy angażuje wiele znanych osób. Z orkiestrą Juka Owsiaka gra też Agnieszka Woźniak-Starak. Co przekazała na aukcję?
lead_text_int:
  Nadchodzący finał Wielkiej Orkiestry Świątecznej Pomocy angażuje wiele znanych osób. Z orkiestrą Juka Owsiaka gra też Agnieszka Woźniak-Starak. Co przekazała na aukcję?
link:
  https://www.plotek.pl/plotek/7,154063,28020376,agnieszka-wozniak-starak-wspiera-wosp-przekazalam-na-aukcje.html
text:
  Wielkimi krokami zbliża się 30. Finał Wielkiej Orkiestry Świątecznej Pomocy. Wiele gwiazd angażuje się we wsparcie akcji. Dołączyła do nich także Agnieszka Woźniak-Starak. Postanowiła oddać na licytacje to, co kocha - torebki. Zwycięzca zgarnie cały zestaw. Według celebrytki to same perełki. Zobaczcie! Informacją o przekazaniu przedmiotów dla WOŚP Woźniak-Starak podzieliła się na Inst

In [14]:
print(f'title:\n  {plotek_dataframe.iloc[99]["title"]}')
print(f'lead_text:\n  {plotek_dataframe.iloc[99]["lead_text"]}')
print(f'link:\n  {plotek_dataframe.iloc[99]["link"]}')
print(f'text:\n  {plotek_dataframe.iloc[99]["text"]}')
print(f'when_published:\n  {plotek_dataframe.iloc[99]["when_published"]}')

title:
  Marta Manowska i brat Pawła Bodziannego na wspólnych wakacjach. Pojawiły się nowe dowody. Już się tak nie ukrywają
lead_text:
  Marta Manowska należy do grona gwiazd, które nie ujawniają szczegółów na temat życia prywatnego. Internauci uważają, że prowadząca show TVP i brat uczestnika "Rolnik szuka żony" są parą. Na ich profilach w mediach społecznościowych pojawiły się nowe dowody, dzięki którym fani mają pewność, że wspólnie wybrali się w podróż.
link:
  https://www.plotek.pl/plotek/7,154063,27985484,marta-manowska-i-brat-pawla-bodziannego-na-wspolnych-wakacjach.html
text:
  Marta Manowska na Instagramie relacjonuje podróż po Meksyku. Prowadząca "Rolnik szuka żony" i brat Pawła Bodziannego od kilkunastu dni dodają na Instagramie podobne relacje, zdjęcia i opisują te same wydarzenia, co nie umknęło uwadze spostrzegawczych internautów. Fani nie mają żadnych wątpliwości, że tych dwoje połączyło gorące uczucie i pasja do podróżowania. Teraz mają nowe dowody potwierdzające, że Ma